In [37]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


from cuml.cluster import HDBSCAN

import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


from visualize import visualize
import os

Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
root_dir = '/media/cat/4TB/donato/'
animal_id = 'DON-006084'
session = '20210519'

data = np.load(os.path.join(root_dir, 
                            animal_id,
                            session,
                            'suite2p',
                            'plane0',
                            'binarized_traces.npz'))
X = data['binarized_peak'].T

# 
pca = PCA(n_components=100)
X_pca = pca.fit_transform(X)

pc1 = 0
pcs = np.arange(pc1,pc1+3,1)
#pcs = np.arange(9,1)
X_pca = X_pca[:,pcs]
print (X_pca.shape)


(55740, 3)


In [26]:
###########################################
############ RUN GPU DBSCAN ###############
###########################################

min_duration = 30
model = HDBSCAN(
                min_cluster_size=min_duration, 
                gen_min_span_tree=True,
                cluster_selection_epsilon=0.05,
               )

# 
db = model.fit_predict(X_pca)

print (np.unique(db))
X_clean = X_pca.copy()

db_clean = db.copy()
for k in range(np.max(db_clean)):
    idx = np.where(db_clean==k)[0]
    #print (k, idx.shape)
    # min duration 
    if idx.shape[0]<min_duration:
        
        X_clean = np.delete(X_clean, idx, axis=0)
        db_clean = np.delete(db_clean, idx, axis=0)

        
# remove noise clusters
idx = np.where(db_clean==-1)[0]
print ("Noise shape: ", idx.shape)
X_clean = np.delete(X_clean, idx, axis=0)
db_clean = np.delete(db_clean, idx, axis=0)

# 
db_ordered = db_clean.copy()
ctr=0
rdn = np.arange(np.unique(db_clean).shape[0])
rdn = np.random.choice(rdn,rdn.shape[0],replace=False)
for k in np.unique(db_clean):
    idx = np.where(db_clean==k)[0]
    db_ordered[idx] = rdn[ctr]
    ctr+=1

Label prop iterations: 20
Label prop iterations: 9
Label prop iterations: 11
Label prop iterations: 6
Label prop iterations: 2
Iterations: 5
13614,162,2539,25,294,2056
Label prop iterations: 3
Iterations: 1
1862,52,302,8,55,102
[-1  0  1  2  3  4  5  6  7  8  9 10 11 12]
Noise shape:  (3216,)


In [45]:
###########################################
############ RUN GPU DBSCAN ###############
###########################################

print (X_clean.shape, db.shape)
v = visualize.Visualize()

size = 1
fig = v.plot_scatter_3d( 
                       X_pca[:,:3], 
                       np.ones(X_pca.shape[0]), 
                       'clusters',
                        'Greys',
                       1,
                       1,
                       None,
                       size)

# 
size = 3
fig = v.plot_scatter_3d( 
                       X_clean[:,:3], 
                       db_ordered, 
                       'clusters',
                        'Edge',
                        1,1,
                        fig,
                        size)
#fig.show()
fig.update_layout(
#     font_family="Courier New",
#     font_color="blue",
#     title_font_family="Times New Roman",
#     title_font_color="red",
#     legend_title_font_color="green",
    title = animal_id + " " + session + ";  PCs: "+str(pcs)+ 
    ";  allpoints (grey): "+ str(X_pca.shape[0])+ 
    ";  clustered points (colors): "+str(X_clean.shape[0])
)
#fig.show()
import plotly.io as pio
fname_out = "/home/cat/code/manifolds/docs/pca_"+animal_id + "_" + session + "_PCs_"+str(pcs)+".html"
pio.write_html(fig, file=fname_out, auto_open=True)

#fig.write_html(")

(52524, 3) (55740,)
